### Calls our final API with a sample of our trainings data and generates metricks

In [1]:
import os
import requests
import pandas as pd
from PIL import Image
from datetime import datetime
import io
import random
# Initialize an empty dataframe
df = pd.DataFrame(columns=['labels','pred_labels', 'con_nowildfire', 'con_wildfire', 'start_time', 'end_time'])

# Directories
base_dir = '../data/test'
directories = [os.path.join(base_dir, d) for d in ['nowildfire', 'wildfire']]

for directory in directories:
    sample = random.sample(os.listdir(directory), 500) # select 500 images as samples for each directory
    for filename in sample:
        if filename.endswith(".jpg"):
            file_path = os.path.join(directory, filename)
            # Open the image
            img = Image.open(file_path)
            img_byte_arr = io.BytesIO()
            img.save(img_byte_arr, format='TIFF')
            img_byte_arr = img_byte_arr.getvalue()        
            # API call
            headers = {'Content-Type': 'image/tiff'}
            url = 'https://staging-yatai.yatai.k8s.eai.dziubalabs.de/predict_image' 
            
            start_time = datetime.now() # timestamp before API call
            
            # Send image
            response = requests.post(url,data=img_byte_arr, headers=headers)
            
            end_time = datetime.now() # timestamp after API call
            
            # Parse the response
            result = response.json()
            
            # Prepare the label
            label = 0 if 'nowildfire' in directory else 1
            pred_label = 0 if result[0] > result[1] else 1
            # Append the result to the dataframe
            new_df = pd.DataFrame([{
                'labels': label,
                'pred_labels' : pred_label,
                'con_nowildfire': result[0], 
                'con_wildfire': result[1], 
                'start_time': start_time, 
                'end_time': end_time
            }])
            df = pd.concat([df, new_df], axis=0, ignore_index=True)
            if len(df)%10 == 0:
                print(len(df))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [2]:
df['duration'] = df['end_time'] - df['start_time']

In [13]:
df.to_csv('sample_test_inference.csv')

In [12]:
df

,con_nowildfire,con_wildfire,start_time,end_time,labels,pred_labels,duration
0,0.9901782,0.009821784,2023-07-17 16:59:59.316771,2023-07-17 17:00:00.446583,0.0,0.0,0 days 00:00:01.129812
1,0.003060331,0.99693966,2023-07-17 17:00:00.460576,2023-07-17 17:00:01.752738,0.0,1.0,0 days 00:00:01.292162
2,0.9384478,0.061552245,2023-07-17 17:00:01.769738,2023-07-17 17:00:03.127891,0.0,0.0,0 days 00:00:01.358153
3,0.00025165087,0.9997483,2023-07-17 17:00:03.138791,2023-07-17 17:00:04.957762,0.0,1.0,0 days 00:00:01.818971
4,0.5776911,0.42230895,2023-07-17 17:00:04.968076,2023-07-17 17:00:07.111969,0.0,0.0,0 days 00:00:02.143893
...,...,...,...,...,...,...,...
994,9.3948234e-05,0.99990606,2023-07-17 17:41:27.382609,2023-07-17 17:41:28.322051,1.0,0.0,0 days 00:00:00.939442
995,0.00013332158,0.9998666,2023-07-17 17:41:28.332619,2023-07-17 17:41:52.726122,1.0,1.0,0 days 00:00:24.393503
996,5.109928e-05,0.99994886,2023-07-17 17:41:52.753569,2023-07-17 17:41:53.716496,1.0,0.0,0 days 00:00:00.962927
997,0.00014001512,0.9998599,2023-07-17 17:41:53.734013,2023-07-17 17:41:54.833560,1.0,1.0,0 days 00:00:01.099547


In [11]:
df = df.drop(columns=['label', 'pred_label'])

In [4]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [6]:
accuracy = accuracy_score(df['labels'], df['pred_labels'])
recall = recall_score(df['labels'], df['pred_labels'], average='macro')
precision = precision_score(df['labels'], df['pred_labels'], average='macro')
f1 = f1_score(df['labels'], df['pred_labels'], average='macro')
avg_inference = df['duration'].mean()
print('Accuracy: ', accuracy)
print('Recall: ', recall)
print('Precision: ', precision)
print('F1 Score: ', f1)
print('Average inference time ', avg_inference)

Accuracy:  0.4874874874874875
Recall:  0.4874308617234469
Precision:  0.4872679734316384
F1 Score:  0.485813549507817
Average inference time  0 days 00:00:02.503168487
